In [ ]:
import jax.numpy as jnp
from scipy import signal
import numpy as np
import time

import sys
sys.path.append('./processors')
import fir_filter, iir_filter, clip, delay_line, lowpass_feedback_comb_filter as lbcf, allpass_filter, freeverb, serial_processors
from train import train, evaluate, process

### TODO
* rename git:khiner/jax_audio_experiments to git:khiner/python_audio_processors with a `/processors/jax` directory
* copy all (non-jax) audio processors in git:khiner/jupyter_notebooks here
* publish `python_audio_processors` as a public library
* delete all (now duplicated) processors in git:khiner/jupyter_notebooks, replace with `python_audio_processors` imports
* move all existing analysis into a new `/analysis` directory here

In [ ]:
from IPython.display import Audio
from scipy.io.wavfile import read as readwav

# sample_rate = 44_100
# X = np.random.randn(sample_rate * 40)
sample_rate, X = readwav('speech-male.wav')
Audio(X, rate=sample_rate)

In [ ]:
tail_length = 4 * sample_rate # let it ring out
X = np.concatenate([X, np.zeros(tail_length)])

presets = {
    'flat_space': {
        'wet': 0.3,
        'dry': 0.7,
        'width': 0.5,
        'damp': 0.35,
        'room_size': 1.04,
    },
    'expanding_space': {
        'wet': 0.33,
        'dry': 0.0,
        'width': 0.5,
        'damp': 0.2,
        'room_size': 1.073,
    },
}

processor = freeverb
audio_for_preset = {preset_name: processor.tick_buffer({'params': params, 'state': processor.init_state()}, X) for preset_name, params in presets.items()}

In [ ]:
output_for_preset = {preset_name: Audio(audio, rate=sample_rate) for preset_name, audio in audio_for_preset.items()}

In [ ]:
output_for_preset['flat_space']

In [ ]:
output_for_preset['expanding_space']

## TODO
* demonstrate single-sample tick
* plot processing time normalized to real_time = 1.0
* implement streaming processing (chunked)
* compare to C++ performance
* provide real-time control over parameters via js
  - just in jupyter for now
* perform inference real-time, displaying estimated parameter values via auto-controlled, disabled knobs
  - interface idea: two separate reverb controllers, on top and bottom of screen, each with a knob for each param. You can mute or solo either reverb separately. At any point, you can hit the 'learn' button for either reverb. This will disable its knobs, and will then listening to the input & output audio of the other (still user-controllable) reverb and live-update its own parameters to try and match the other reverb parameter values.
* check if URL is available: python_audio_processors.io
  - otherwise, just use karlhiner.com/python_audio_processors
* port to ^
* charts for impulse response, magnitude spectrogram and phase, updating in real-time